In [1]:
print("test")

test


In [5]:
import re
sample_file_name = "../dutch-language/data/sample-sentences-nlwiki.txt"
short_file_name = "../dutch-language/data/short-sentences-nlwiki.txt"
full_file_name = "../dutch-language/data/clean-sentences-nlwiki.txt"
corrected_file_name = "../dutch-language/data/corrected-sentences-nlwiki.txt"

file_name = corrected_file_name

regex_worden = re.compile(r'(.+ z|Z)end\b.*')
with open(file_name) as file:
    total_count = 0
    match_count = 0

    for line in file:
        if (total_count >= 1000000):
            break
        total_count += 1
        clean_line = line.rstrip()
        if (regex_worden.match(clean_line)):
            match_count += 1
            if (total_count > 500000):
                print(clean_line)

    print(total_count)
    print(match_count)

Voor communicatie werd het zend- en ontvangstation Radio Malabar gebruikt, dat intercontinentaal telefoneren mogelijk maakte met Nederland.
PZB bestaat uit een passieve LC-kring naast het spoor bij het sein en een actieve zend/ontvangspoel aan de trein.
in Openbaringen 1 schrijft Johannes het volgende: Hetgeen gij ziet, schrijf dat in een boek en zend het aan de zeven gemeenten.
De zend- en ontvangstapparatuur is meestal ondergebracht in een gebouw, kast of container.
RTL 4 zend het programma in december 2019 en januari 2020 uit.
Een oplossing is dan een digitale hoofdtelefoon die in en rond de 2,4 GHz ontvangt en zend.
En in het heldendicht Hákonarmál is het Hákon de Goede die naar Walhalla wordt gevoerd door de walkure Göndul en Odin zend Hermóðr en Bragi om hem te begroeten.
Val binnen als de bewoners om hulp vragen, neem het land in, zend kolonisten naar het gebied of de vorst moet er zelf gaan wonen.
1000000
21


In [1]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
    "roberta", "outputs/RoBERTa-001", use_cuda=True
)
print(type(model))


<class 'simpletransformers.classification.classification_model.ClassificationModel'>


In [6]:
predictions, raw_outputs = model.predict([
    "Ik word nieuwsgierig.",
    "Hoe word je gevraagd?",
    "Wat wordt je gevraagd?",
    "Wat word je opdringerig, zeg!",

    "Ik zend een pakje.",
    "Ik zend je een pakje.",
    "Ik zend u een heel mooie trui.",
    "Je zendt het toch nog vandaag op, hoop ik.", #
    "Jij zendt een hele grote doos naar oma.",
    "Zend je dat straks uit?", #
    "Zend jij echt die fiets via de post?",
    "Welke baas zendt je nu helemaal naar London voor 1 klant?",

    # With spelling mistake
    "Ik wordt nieuwsgierig.",
    "Hoe wordt je gevraagd?",
    "Wat word je gevraagd?",
    "Wat wordt je opdringerig, zeg!",

    "Ik zendt een pakje.",
    "Ik zendt je een pakje.",
    "Ik zendt u een heel mooie trui.", #
    "Je zend het toch nog vandaag op, hoop ik.",
    "Jij zend een hele grote doos naar oma.",
    "Zendt je dat straks uit?", #
    "Zendt jij echt die fiets via de post?",
    "Welke baas zend je nu helemaal naar London voor 1 klant?",

    # Trying other words
    # Correct spelling
    "Ik vind dit toch niet zo mooi.",
    "Wat vind jij van al die aandacht?",
    "Hoe vind ik nu de ingang?",
    "Het is toch erg dat hij dat niet vindt.",
    "Hoe zwaar vindt hij de opleiding?",

    "Ik loop er zo maar voorbij.",
    "Loop jij ook zo snel?",
    "Je loopt daar beter niet telkens over.",
    "Jij loopt echt helemaal naar zee?",
    "En daarom loopt hij er met een grote bocht omheen.",

    # With spelling mistake
    "Ik vindt dit toch niet zo mooi.",
    "Wat vindt jij van al die aandacht?",
    "Hoe vindt ik nu de ingang?",
    "Het is toch erg dat hij dat niet vind.", # Incorrectly evaluated to 0"
    "Hoe zwaar vind hij de opleiding?",

    "Ik loopt er zo maar voorbij.",
    "Loopt jij ook zo snel?",
    "Je loop daar beter niet telkens over.",
    "Jij loop echt helemaal naar zee?",
    "En daarom loop hij er met een grote bocht omheen."
])
print()
print(predictions)
print(raw_outputs)

# Expected [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
# Actual   [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1]



INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 6/6 [00:01<00:00,  4.20it/s]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1
 0 1 1 1 1 1 1]
[[ 4.915488   -4.789471  ]
 [ 4.839436   -4.840326  ]
 [ 4.8377805  -4.830837  ]
 [ 4.910914   -4.896868  ]
 [ 4.8765435  -4.702445  ]
 [ 4.861317   -4.7078547 ]
 [ 4.8241663  -4.7353034 ]
 [ 4.830417   -4.747499  ]
 [ 4.7849355  -4.7121396 ]
 [ 4.729992   -4.657965  ]
 [ 4.7955823  -4.7781553 ]
 [ 4.705295   -4.7266016 ]
 [-4.787341    5.1297984 ]
 [-4.793274    5.018534  ]
 [-4.627256    4.660578  ]
 [-4.6352367   4.9608326 ]
 [-4.767566    5.138868  ]
 [-4.7389708   5.115666  ]
 [-4.737144    5.124101  ]
 [-4.6974187   4.7999873 ]
 [-4.7432747   4.797371  ]
 [-4.758967    5.077504  ]
 [-4.752236    5.053839  ]
 [-4.560308    4.6208158 ]
 [ 4.019045   -3.9757798 ]
 [ 4.4000516  -4.088637  ]
 [ 4.4310527  -4.48809   ]
 [ 3.8491602  -3.70643